In [1]:
\l C:\q\columbiaHdb

In [2]:
/ we will remove 2019.01.09 from our analysis
/ i below is the virtual column; you can think of it as the row numbers
/ see "virtual column i" in "https://code.kx.com/q/basics/qsql/"

`x xasc select count i by date from bin10

date      | x      
----------| -------
2019.01.09| 29058  
2019.12.24| 610324 
2019.11.29| 616629 
2019.07.03| 626717 
2019.09.18| 887893 
2019.03.08| 1122500
2019.12.10| 1133044
2019.12.11| 1133044
2019.12.12| 1133044
2019.12.13| 1133044
2019.12.16| 1133044
2019.12.17| 1133044
2019.12.18| 1133044
2019.12.19| 1133044
2019.12.20| 1133044
2019.12.23| 1133044
2019.12.26| 1133044
2019.12.27| 1133044
2019.12.09| 1135385
2019.12.06| 1139580
..


In [3]:
/ set console size
\c 50 200

In [4]:
/ remove 2019.01.09
selectedDates: date except 2019.01.09

# Question 1
Wrap the script from Week 5 into a function of dt, the date.

In [5]:
cleanData:{[dt]
 tbl: select from bin10 where date = dt;
 update reverse fills reverse date, 
 reverse fills reverse mid, 
 reverse fills reverse spread, 
 reverse fills reverse vol, 
 reverse fills reverse adv 
 by id from tbl}

In [6]:
computeImpact : {[trade; adv; vol; h]
 / exponential decay constant per 10 seconds, 
 / i.e. \beta * \delta t in the formula above
 beta: (log 2) % 6 * h; 
 lambda: 1;
 dailyVol: vol * sqrt 6 * 60 * 6.5;
 / initialize impact to be zero
 / drop the first elemetn from ema in the output
 1_ema[beta; 0, lambda * dailyVol * trade % beta * adv]}; 

In [7]:
computeImpactCov:{[dt] 
 tbl: cleanData dt;
 crossTbl: ([]h: 1,5, 30, 60) cross tbl;
 tbl2: update I: computeImpact[trade; adv; vol; first h] by id, h from crossTbl;
 crossTbl2: ([]DeltaT:1,15,60) cross tbl2;
 tbl3: update rtn:(neg 1)+mid%(6*DeltaT) xprev mid, DeltaI: I + neg (6*DeltaT) xprev I by DeltaT, h, id from crossTbl2;
 / return an unkeyed table (which is helpful for raze)
 0!select first date, covI: sum rtn*DeltaI, varI: sum DeltaI*DeltaI, varrtn: sum rtn*rtn by DeltaT, h, id from tbl3};

In [8]:
computeImpactCov 2019.01.03

DeltaT h id date       covI           varI          varrtn     
---------------------------------------------------------------
1      1 0  2019.01.03 6.682889e-005  7.761975e-006 0.002917216
1      1 1  2019.01.03 1.871033e-005  2.092999e-006 0.001763429
1      1 2  2019.01.03 3.39635e-005   4.69036e-006  0.00233163 
1      1 3  2019.01.03 3.305e-005     7.10511e-006  0.001962249
1      1 4  2019.01.03 1.85482e-005   4.324463e-006 0.00132793 
1      1 5  2019.01.03 4.325126e-005  5.366103e-006 0.003459075
1      1 6  2019.01.03 3.069263e-005  3.523943e-006 0.003358983
1      1 7  2019.01.03 3.325452e-005  3.988708e-006 0.002348954
1      1 8  2019.01.03 0.0001086091   1.568785e-005 0.006361805
1      1 9  2019.01.03 4.362178e-005  7.813313e-006 0.005673803
1      1 10 2019.01.03 4.957895e-005  7.301048e-006 0.006336665
1      1 11 2019.01.03 1.197273e-005  2.232379e-006 0.001589386
1      1 12 2019.01.03 2.174861e-005  6.963183e-006 0.001065524
1      1 13 2019.01.03 3.613895e-005  3.

# Question 2
Loop over values of dt and aggregate variances and covariances into a single table across dates, stocks, halflives, and prediction horizons.


In [9]:
summaryStats: raze computeImpactCov peach selectedDates

In [10]:
/ set working directory
\cd C:\Users\longz\OneDrive\Desktop\Price Impact Models\Week 6 Assignmnet

/ save the data in a csv file
/ you can load the data without running the peach command next time
save `summaryStats.csv

`:summaryStats.csv


In [11]:
summaryStats

DeltaT h id date       covI          varI          varrtn      
---------------------------------------------------------------
1      1 0  2019.01.02 3.01604e-005  5.321394e-006 0.002523996 
1      1 1  2019.01.02 1.970623e-005 4.287721e-006 0.001790285 
1      1 2  2019.01.02 2.46948e-006  2.966949e-006 0.002429288 
1      1 3  2019.01.02 2.840621e-005 3.832767e-006 0.003322996 
1      1 4  2019.01.02 1.606366e-005 4.029094e-006 0.001247307 
1      1 5  2019.01.02 2.487197e-005 3.641015e-006 0.002815581 
1      1 6  2019.01.02 1.758999e-005 2.022091e-006 0.001696959 
1      1 7  2019.01.02 6.917314e-005 9.636768e-006 0.006796038 
1      1 8  2019.01.02 2.342327e-005 5.425924e-006 0.001499933 
1      1 9  2019.01.02 4.437051e-005 2.205153e-005 0.003664438 
1      1 10 2019.01.02 1.665177e-005 3.983575e-006 0.00113871  
1      1 11 2019.01.02 3.299057e-005 6.763197e-006 0.003650561 
1      1 12 2019.01.02 4.787888e-005 8.542749e-006 0.003143918 
1      1 13 2019.01.02 4.75606e-005  1.5

Questions 3 and 4 concern about the regression $ r = \lambda \Delta I + \epsilon$, where $\epsilon$ is assumed to be Gaussian and i.i.d.

# Question 3

Compute $\lambda = E[r \Delta I]/E[(\Delta I)^2]$ and in-sample $R^2$ by stock, halflife, and prediction horizon; that is, run a regression using the data across all dates for each (stock, halflife, prediction horizon) pair. Plot the corresponding distribution; cf.\ page 36 of Lecture 5.

In [12]:
tblQ3: update lambda: (sum covI) % sum varI, 
       R2: (sum covI) * (sum covI) % (sum varI) * sum varrtn 
 by id, h, DeltaT from summaryStats

In [13]:
tblQ3: select lambda: last lambda, R2: last R2 by id, h, DeltaT from tblQ3
save `tblQ3.csv
tblQ3

`:tblQ3.csv


id h  DeltaT| lambda   R2         
------------| --------------------
0  1  1     | 4.029894 0.05554571 
0  1  15    | 3.503654 0.005701841
0  1  60    | 4.089801 0.002426374
0  5  1     | 5.043827 0.1040647  
0  5  15    | 4.796547 0.04331196 
0  5  60    | 4.722691 0.01634841 
0  30 1     | 5.1927   0.1158953  
0  30 15    | 4.532993 0.08510361 
0  30 60    | 3.707241 0.05499832 
0  60 1     | 5.181921 0.1165311  
0  60 15    | 4.260502 0.08693151 
0  60 60    | 3.341161 0.06597691 
1  1  1     | 4.446895 0.06115941 
1  1  15    | 4.267403 0.008358513
1  1  60    | 5.901814 0.004944258
1  5  1     | 5.178476 0.1035574  
1  5  15    | 5.049616 0.04807694 
1  5  60    | 5.92922  0.0249152  
1  30 1     | 5.133855 0.110441   
1  30 15    | 4.656732 0.09167812 
1  30 60    | 4.168303 0.06671141 
1  60 1     | 5.094629 0.1103784  
1  60 15    | 4.345486 0.0928042  
1  60 60    | 3.579131 0.07324173 
2  1  1     | 4.680557 0.06683823 
2  1  15    | 3.987683 0.006436635
2  1  60    | 4.6634

# Question 4

Compute $\lambda = E[r \Delta I]/E[(\Delta I)^2]$ and out-of-sample $R^2$ by month, halflife, and prediction horizon; that is run a regression using the data across all stocks for each (month, halflife, prediction horizon) pair and calculate the model $R^2$ in the next month. Plot the corresponding timeseries; cf.\ page 37 of Lecture 5.

Out-of-sample $R^2$ for $y \sim x$ is

$$
1 - \frac{\sum_{i=1}^{n}(y_i - \lambda x_i)^2 }{\sum_{i=1}^{n} y_i^2} = \frac{2\lambda \sum_{i=1}^{n} x_i y_i - \lambda^2 \sum_{i=1}^{n} x_i^2}{\sum_{i=1}^{n} y_i^2}.
$$

In our case, $y$ is $r$ and $x$ is $\Delta I$.

In [14]:
tblQ4: select sum varI, sum covI, sum varrtn, lambda: (sum covI) % sum varI by date.month, h, DeltaT from summaryStats
tblQ4: update prevLambda: prev lambda by h, DeltaT from tblQ4
tblQ4

month   h  DeltaT| varI       covI      varrtn   lambda   prevLambda
-----------------| -------------------------------------------------
2019.01 1  1     | 0.05322332 0.2482827 19.89801 4.664924           
2019.01 1  15    | 0.08959574 0.307477  225.3182 3.431826           
2019.01 1  60    | 0.07981461 0.2958939 673.3194 3.707265           
2019.01 5  1     | 0.06395738 0.3683624 19.89801 5.759497           
2019.01 5  15    | 0.3763849  1.747806  225.3182 4.643667           
2019.01 5  60    | 0.4250236  1.65614   673.3194 3.896583           
2019.01 30 1     | 0.06766657 0.3971312 19.89801 5.868942           
2019.01 30 15    | 0.8592862  3.613265  225.3182 4.20496            
2019.01 30 60    | 2.44328    6.878296  673.3194 2.815189           
2019.01 60 1     | 0.06848723 0.3999012 19.89801 5.839062           
2019.01 60 15    | 1.007664   3.914097  225.3182 3.884327           
2019.01 60 60    | 3.727777   9.080889  673.3194 2.436006           
2019.02 1  1     | 0.03260784 0.15

In [15]:
tblQ4: update R2: ((2 * prevLambda * covI) + neg (prevLambda * prevLambda * varI)) % varrtn 
 by month h, DeltaT from tblQ4
save `tblQ4.csv
tblQ4

`:tblQ4.csv


month   h  DeltaT| varI       covI      varrtn   lambda   prevLambda R2         
-----------------| -------------------------------------------------------------
2019.01 1  1     | 0.05322332 0.2482827 19.89801 4.664924                       
2019.01 1  15    | 0.08959574 0.307477  225.3182 3.431826                       
2019.01 1  60    | 0.07981461 0.2958939 673.3194 3.707265                       
2019.01 5  1     | 0.06395738 0.3683624 19.89801 5.759497                       
2019.01 5  15    | 0.3763849  1.747806  225.3182 4.643667                       
2019.01 5  60    | 0.4250236  1.65614   673.3194 3.896583                       
2019.01 30 1     | 0.06766657 0.3971312 19.89801 5.868942                       
2019.01 30 15    | 0.8592862  3.613265  225.3182 4.20496                        
2019.01 30 60    | 2.44328    6.878296  673.3194 2.815189                       
2019.01 60 1     | 0.06848723 0.3999012 19.89801 5.839062                       
2019.01 60 15    | 1.007664 